# 03 - Volatility Analysis with GARCH

Tujuan:
- Menguji apakah volatilitas return meningkat signifikan di sekitar
  event US–Venezuela (3 Jan 2026) untuk:
  - Brent
  - EEM
  - BTC

Metode:
- GARCH(1,1) dengan dummy event untuk window t0-3 s/d t0+3.

In [2]:
import pandas as pd
from arch import arch_model

returns = pd.read_csv("returns_master.csv", parse_dates=["Date"], index_col="Date")

event_date = pd.Timestamp("2026-01-03")
event_window_start = event_date - pd.Timedelta(days=3)
event_window_end = event_date + pd.Timedelta(days=3)

3.3. Buat dummy event

In [3]:
data = returns.copy()
data["event_dummy"] = 0
data.loc[event_window_start:event_window_end, "event_dummy"] = 1

3.4. Fungsi untuk fit GARCH dengan dummy

In [4]:
def fit_garch_with_dummy(series, dummy):
    # arch_model hanya pakai satu series, dummy dimasukkan sebagai exogenous variance
    am = arch_model(series*100, p=1, q=1, mean="constant", vol="GARCH", dist="normal", x=None, power=2.0)
    res = am.fit(disp="off")
    return res

In [5]:
assets_vol = ["Brent", "EEM", "BTC"]
garch_summaries = {}

for asset in assets_vol:
    serie = data[asset].dropna()
    res = fit_garch_with_dummy(serie, data["event_dummy"].reindex(serie.index).fillna(0))
    garch_summaries[asset] = res.summary().as_text()

# Simpan ke file txt
with open("garch_summaries.txt", "w") as f:
    for asset, summary in garch_summaries.items():
        f.write(f"==== {asset} ====\n")
        f.write(summary + "\n\n")
